<h3> 1. ﻿naver finance에서 코스피 200의 리스트를 받아라</h3>

In [ ]:
!pip install pandas-ta

In [41]:
from bs4 import BeautifulSoup
import requests as rq
from datetime import datetime                # 객체를 date 타입으로 만들어줄 모듈
import pandas as pd
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
import time
import urllib
import re
import lxml
import pandas_ta as pta

In [2]:
jongmoks = [] #코스피200의 종목이름
codes = [] #코스피200의 종목코드
for page in range(1,21):
    url = f'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'
    html = rq.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        #print(item)
        txt = item.a.get('href') 
        k = re.search('[\d]+', txt) ##정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            codes.append(code)
            name = item.text
            jongmoks.append(name)
data = dict(zip(codes, jongmoks)) #코드와 종목이름 결합한 딕셔너리

<h3> 2. 2022년 코스피 200의 1월 ~3월까지의 주가를 모두 받아라﻿ </h3>

In [24]:
for code in data.keys():
    df = None
    dic1 = {'date':[],
       'price':[]}
    for page in range(17, 23):
        req = rq.get(f'https://finance.naver.com/item/sise_day.naver?code={code}&page={page}', headers={'User-Agent':'Mozilla/5.0'})
        html = BeautifulSoup(req.text, 'html.parser')
        lines = html.find_all('tr', {'onmouseover':'mouseOver(this)'})
        for line in lines:
            date = line.find_all('td', {'align': 'center'})[0]
            price = line.find_all('td', {'class': 'num'})[0]
            date_ = date.text.strip()
            if date_ == "":
                break
#             print(date_)
            price_ = price.text.strip()
            dic1['date'].append(date_)
            dic1['price'].append(price_)
    
    df = pd.DataFrame(dic1)
#     print(df['date'])
    df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'], errors = 'coerce')
    df.loc[:,'price'] = df.loc[:,'price'].str.replace(",","")
#     print(df['price'])
    df.loc[:,'price'] = pd.to_numeric(df.loc[:,'price'])
    df1 = df[(df.loc[:,'date'] >= '2022-01-01') & (df.loc[:,'date'] <= '2022-03-31')]
    df2 = df1.sort_values(by='date')
    globals()['df_{}'.format(code)] = df2.reset_index(drop = True)
    

<h3> 3. 그들을 DataFrame화 하여 각각에 대해 기술적 분석, MA, MACD, RSI를 구하라 </h3>

In [44]:
for code in codes:
    # 1월부터 3월까지 공휴일이 아닌 일 수는 58일이기 때문에 5일, 20일에 대한 이동평균만 구한다.
    #단순이동평균 SMA
    globals()['df_{}'.format(code)]['sma5'] = globals()['df_{}'.format(code)]['price'].rolling(window=5).mean()
    globals()['df_{}'.format(code)]['sma20'] = globals()['df_{}'.format(code)]['price'].rolling(window=20).mean()
    
    #지수이동평균 EMA
    globals()['df_{}'.format(code)]['ema5'] = globals()['df_{}'.format(code)]['price'].ewm(5).mean()
    globals()['df_{}'.format(code)]['ema20'] = globals()['df_{}'.format(code)]['price'].ewm(20).mean()
    
    #MACD, MACD Signal
    exp1 = globals()['df_{}'.format(code)]['price'].ewm(span=12, adjust=False).mean()
    exp2 = globals()['df_{}'.format(code)]['price'].ewm(span=26, adjust=False).mean()
    globals()['df_{}'.format(code)]['MACD'] = exp1 - exp2
    globals()['df_{}'.format(code)]['Signal line'] = globals()['df_{}'.format(code)]['MACD'].ewm(span=9, adjust=False).mean()
    globals()['df_{}'.format(code)]['rsi14'] = pta.rsi(globals()['df_{}'.format(code)]['price'], length = 14)
    

In [47]:
df_051910

,date,price,sma5,sma20,ema5,ema20,MACD,Signal line,rsi14
0,2022-01-03,618000,NaN,NaN,618000.000000,618000.000000,0.000000,0.000000,NaN
1,2022-01-04,644000,NaN,NaN,632181.818182,631317.073171,2074.074074,414.814815,NaN
2,2022-01-05,664000,NaN,NaN,644769.230769,642747.026170,5270.866308,1386.025113,NaN
3,2022-01-06,690000,NaN,NaN,659329.359165,655438.357356,9789.482829,3066.716657,NaN
4,2022-01-07,719000,667000.0,NaN,675956.568480,669420.393145,15531.538704,5559.681066,NaN
5,2022-01-10,710000,685400.0,NaN,684487.448036,677034.575430,19135.354120,8274.815677,NaN
6,2022-01-11,734000,703400.0,NaN,695934.086844,686410.531937,23655.319427,11350.916427,NaN
7,2022-01-12,773000,725200.0,NaN,712670.839440,699169.845784,30038.137608,15088.360663,NaN
8,2022-01-13,755000,738200.0,NaN,721421.668910,706650.558986,33260.708689,18722.830268,NaN
9,2022-01-14,716000,737600.0,NaN,720344.009397,707803.697447,32295.364566,21437.337128,NaN


<h3> 4. 이들을 csv로 만들어서 코드와 csv 200개를 알집으로 제출해라</h3>